# CloakHat Patch Generation Pipeline

## 1: Conda Environment Setup

`conda create -n cloakhat python=3.10 -y` <br>
`conda activate cloakhat`

PyTorch with CUDA <br>
`conda install pytorch torchvision pytorch-cuda=11.8 -c pytorch -c nvidia -y`

PyTorch3D for differentiable rendering <br>
`pip install "git+https://github.com/facebookresearch/pytorch3d.git"`

Detection models <br>
`pip install ultralytics`

Pip stuff <br>
`pip install opencv-python-headless matplotlib tqdm tensorboard pyyaml trimesh`

## 2: Python Setup

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
from datetime import datetime
import logging

from pytorch3d.io import load_obj
from pytorch3d.structures import Meshes
from pytorch3d.renderer import (look_at_view_transform, FoVPerspectiveCameras, RasterizationSettings, MeshRenderer, MeshRasterizer, SoftPhongShader, TexturesUV, PointLights)

from ultralytics import YOLO

logging.basicConfig(level=logging.INFO, format='%(asctime)s | %(message)s')
logger = logging.getLogger(__name__)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
logger.info(f"Device: {device}")

Path(CONFIG['output_dir']).mkdir(parents=True, exist_ok=True)

## 3: Config

In [ ]:
CONFIG = {
    'dataset_dir': './data/drone_footage', #Drone footage
    'mesh_path': './assets/hat.obj', #Hat meshes
    'output_dir': './outputs',
    
    #Generator
    'latent_channels': 128,
    'latent_size': 9, #Spatial size of latent input
    'texture_size': 324, #Output texture size from generator
    
    # Viewpoint sampling (aerial)
    'elevation_range': (60, 90), #Degrees from horizontal (90 = overhead)
    'scale_range': (0.3, 1.2), #Altitude proxy
    
    #Training Stage 1
    'stage1_epochs': 100,
    'stage1_batch_size': 8,
    'stage1_lr': 2e-4,
    
    #Training Stage 2  
    'stage2_iterations': 2000,
    'stage2_lr': 0.01,
    'local_latent_size': 18, #Size of optimizable latent pattern
    
    #Loss weights
    'lambda_tv': 2.5, # Total variation
    'lambda_nps': 0.01, # Non-printability score
    'lambda_info': 0.1, # Mutual information (Stage 1 only)
    
    #T-SEA Stuff
    'cutout_prob': 0.9,
    'cutout_ratio': 0.4,
    'shakedrop_prob': 0.5,
    
    #Rendering
    'render_size': 256,
    
    #Printing (PLACEHOLDER)
    'nps_threshold': 0.7,  # Saturation * brightness threshold
    
    #Attack config: 'white', 'gray', or 'black'
    'attack_mode': 'gray',
}